### Inci's Preliminary Dispersion Calculation Code

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from g4beam import *
from scan import *
from scipy.optimize import differential_evolution

import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
import numpy as np
import pandas as pd
from tqdm import *
import pickle
import itertools
from tabulate import tabulate
import tempfile
import glob
import json
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
# Make Sure g4bl is here
import os
os.environ["PATH"] += os.pathsep + "/home/incik/G4beamline-3.08/bin"
import shutil
print(shutil.which("g4bl"))

/home/incik/G4beamline-3.08/bin/g4bl


In [3]:
# Load data POST WEDGE as a Dataframe so that you can use Daniel Fu's functions
filename = "particles_after.txt"

# Skip the first two header lines that start with '#'
with open(filename) as f:
    # Read until the line containing column names
    for line in f:
        if line.startswith("#x "):
            columns = line.strip().lstrip("#").split()
            break

# Now load the data into a DataFrame
df = pd.read_csv(filename, comment="#", delim_whitespace=True, names=columns)

x_params, y_params, z_emit = calc_all_params(df) # _params are tuples of the form (emittance, beta, gamma, alpha, D, D')
D_dict = {"D_x": x_params[4], "D'_x": x_params[5], "D_y": y_params[4], "D'_y": y_params[5]}
print(D_dict)
cost = D_dict["D_x"]**2 + D_dict["D'_x"]**2 + D_dict["D_y"]**2 + D_dict["D'_y"]**2 
print(cost)
print(r"Epsilon_z: "+str(z_emit))

{'D_x': np.float64(0.007405947668631706), "D'_x": np.float64(-0.18883973986720437), 'D_y': np.float64(5.278092595079276e-05), "D'_y": np.float64(0.012149181979370281)}
0.03586290082257793
Epsilon_z: 8.303137587240382


/tmp/ipykernel_20585/2940929977.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(filename, comment="#", delim_whitespace=True, names=columns)


In [4]:
# ---------------- USER CONFIG ----------------
G4BEAMLINE_CMD = "g4bl"
TEMPLATE_FILE = "Achromat.g4bl"
OUTPUT_DIR = "achromat_runs"
VD_FILENAME = "vd_achromat.txt"   # virtual detector writes this file (ascii)
N_PARTICLES = 5000             # increase for lower noise
G4BLFILE = f"/home/incik/Cooling_4D/AchromatTest/run.g4bl"
G4BLOUTPUT =f"/home/incik/Cooling_4D/AchromatTest/{VD_FILENAME}"
E0 = 1e4                       # beam energy (GeV) - for a muon collider

# {B1_field}, {B1_width}, {B1_height}, {B1_length}, {B1_z}, {Q1_gradient}, {Q1_length}, {radius_q}
# {B2_field}, {B2_width}, {B2_height}, {B2_length}
# {Drift1_width}, {Drift1_height}, {Drift1_length}, 
# {Drift2_width}, {Drift2_height}, {Drift2_length}

var_names = ["N_PARTICLES", "B1_field", "B1_width", "B1_height", "B1_length", "B1_z",
            "Q1_gradient", "Q1_length", "radius_q", "B2_field", "B2_width", "B2_height", "B2_length",
            "Drift1_width", "Drift1_height", "Drift1_length", "Drift2_width", "Drift2_height", "Drift2_length", "VD_FILENAME"]
# 5000 Particle Opt:
xvec = np.array([int(N_PARTICLES),  -0.0157, 123.8669, 76.5539, 184.5674, 84.8239, -2.7065, 244.3089, 18.1791, -1.4222, 158.1968, 57.5410, 359.0282, 
                 181.6853, 166.1331, 157.8154,  181.9824, 133.1504, 468.2003, VD_FILENAME])
# 1000 Particle Opt:
# xvec = np.array([int(N_PARTICLES),  -0.6719, 63.3749, 156.8672, 224.0275, 63.3824, -91.1134, 156.1167, 14.6261, -1.4102, 109.7400, 197.3407, 274.0035, 175.7871, 100.1926, 71.6003,  76.0153, 73.1403, 725.5903, VD_FILENAME])
# np.array([int(N_PARTICLES), 1.0, 100.0, 100.0, 30.0, 50.0, 12.0, 250.0, 10.0, 4.0, 100.0, 100.0, 200.0, 100.0, 100.0, 250.0, 100.0, 100.0, 250.0, VD_FILENAME])
calcparams = {name: val for name, val in zip(var_names, xvec)}

GAP = 0.1  # in mm (can be up to 1.0 safely)
B1_z_val = float(calcparams["B1_z"])
L_B1 = float(calcparams["B1_length"])
L_D1 = float(calcparams["Drift1_length"])
L_Q1 = float(calcparams["Q1_length"])
L_D2 = float(calcparams["Drift2_length"])
L_B2 = float(calcparams["B2_length"])

Drift1_z = B1_z_val + (L_B1/2) + (L_D1/2) + GAP
Q1_z     = Drift1_z + (L_D1/2) + (L_Q1/2) + GAP
Drift2_z = Q1_z + (L_Q1/2) + (L_D2/2)+ GAP
B2_z     = Drift2_z + (L_D2/2) + (L_B2/2) + GAP
VD_z     = B2_z + (L_B2/2) + 10.0 + GAP

print(B1_z_val, Drift1_z-(L_D1/2), B1_z_val + (L_B1/2))

add_params = {"Drift1_z": Drift1_z, "Q1_z": Q1_z, "Drift2_z":Drift2_z, "B2_z": B2_z, "VD_z": VD_z}
calcparams.update(add_params)

for k, v in calcparams.items():
    if k == "N_PARTICLES" or k == "VD_FILENAME":
        continue
    else:
        calcparams[k] = float(v)

# os.makedirs(OUTPUT_DIR, exist_ok=True)

84.8239 177.20760000000004 177.1076


In [5]:
# Write values to the prepared G4BL template
def write_input_from_template(template_path, out_path, replacements):
    with open(template_path, 'r') as f:
        txt = f.read()
    try:
        txt = txt.format(**replacements)
    except KeyError as e:
        raise RuntimeError(f"Template substitution failed; missing placeholder: {e}")
    with open(out_path, 'w') as f:
        f.write(txt)

# How to Use?
print(calcparams)
write_input_from_template(TEMPLATE_FILE, G4BLFILE, calcparams)

{'N_PARTICLES': np.str_('5000'), 'B1_field': -0.0157, 'B1_width': 123.8669, 'B1_height': 76.5539, 'B1_length': 184.5674, 'B1_z': 84.8239, 'Q1_gradient': -2.7065, 'Q1_length': 244.3089, 'radius_q': 18.1791, 'B2_field': -1.4222, 'B2_width': 158.1968, 'B2_height': 57.541, 'B2_length': 359.0282, 'Drift1_width': 181.6853, 'Drift1_height': 166.1331, 'Drift1_length': 157.8154, 'Drift2_width': 181.9824, 'Drift2_height': 133.1504, 'Drift2_length': 468.2003, 'VD_FILENAME': np.str_('vd_achromat.txt'), 'Drift1_z': 256.11530000000005, 'Q1_z': 457.27745000000004, 'Drift2_z': 813.63205, 'B2_z': 1227.3463000000002, 'VD_z': 1416.9604000000002}


In [6]:
# Run g4bl and get the Output file

result = subprocess.run(["g4bl", G4BLFILE], capture_output=True, text=True, check=True)
print(result)

CompletedProcess(args=['g4bl', '/home/incik/Cooling_4D/AchromatTest/run.g4bl'], returncode=0, stdout="G4BL_DIR=/home/incik/G4beamline-3.08\nG4LEDATA=/home/incik/Geant4Data/G4EMLOW8.0\nG4ENSDFSTATEDATA=/home/incik/Geant4Data/G4ENSDFSTATE2.3\nG4PARTICLEXSDATA=/home/incik/Geant4Data/G4PARTICLEXS4.0\nG4LEVELGAMMADATA=/home/incik/Geant4Data/PhotonEvaporation5.7\nG4RADIOACTIVEDATA=/home/incik/Geant4Data/RadioactiveDecay5.6\nG4beamline Process ID 30982\n\n*************************************************************\n g4beamline version: 3.08                        (Aug 11 2022)\n                      Copyright : Tom Roberts, Muons, Inc.\n                        License : Gnu Public License\n                            WWW : http://g4beamline.muonsinc.com\n\n          ################################\n          !!! G4Backtrace is activated !!!\n          ################################\n\n\n**************************************************************\n Geant4 version Name: geant4-11-00-pat

In [11]:
# Make the results a dataframe and calculate the Courant-Snyder Parameters
df = read_trackfile(G4BLOUTPUT)
x_params, y_params, z_emit = calc_all_params(df)

In [12]:
x_params, y_params, z_emit
D_dict = {"D_x": x_params[4], "D'_x": x_params[5], "D_y": y_params[4], "D'_y": y_params[5]}
print(D_dict)
cost = D_dict["D_x"]**2 + D_dict["D'_x"]**2 + D_dict["D_y"]**2 + D_dict["D'_y"]**2 
print(cost)
print(r"Epsilon_z: "+str(z_emit))

{'D_x': np.float64(-0.01413751725358641), "D'_x": np.float64(-0.01306413688841289), 'D_y': np.float64(0.060572188560189845), "D'_y": np.float64(0.05129255398074614)}
0.006670457187573594
Epsilon_z: 4.3355067704032395
